# BANK ACCOUNT PROJECT

# Step1: Connecting [python-jupyter notebook] with [postgreSQL]

In [1]:
import psycopg2

In [3]:
try:
    conn = psycopg2.connect(
    dbname="my_db",
    user="postgres",
    password="admin",
    host="localhost",  # or your host
    port="5432"  # or your port
    )
except Exception as e:
    print("getting error as :", e)
else:
    print('Connected with postgres successfully')

Connected with postgres successfully


In [5]:
cur= conn.cursor()

In [13]:
conn.rollback()

# Step:2 Creating new table with existing account detail in SQL through python

In [15]:
cur.execute ('''
CREATE TABLE my_project (sl_no NUMERIC(150), account_name VARCHAR(50), account_number NUMERIC(50), account_balance NUMERIC(50));
COMMIT;
''')

In [17]:
cur.execute('''
INSERT INTO my_project (sl_no, account_name, account_number, account_balance)
VALUES (1, 'Pradap',1234,5000),(2, 'Hithesh',1235,5800),(3, 'Ajeeth',1236,5200);
commit;
''')

# Step:3 Creating codes for the bank operation

In [71]:
import psycopg2

class Bank_account:
    def __init__(self,sl_no,account_name,account_number,account_balance=0):
        self.sl_no=sl_no,
        self.account_name=account_name,
        self.account_number= account_number,
        self.account_balance= account_balance
        
        # Establish connection to PostgreSQL
        self.conn = psycopg2.connect(
            dbname="my_db",
            user="postgres",
            password="admin",
            host="localhost",  # or your host
            port="5432"  # or your port
        )
        
        self.cur=self.conn.cursor()
    
        #Check if the account is exist or not
        self.account_exist= self.check_account_exist()

    def check_account_exist(self):
        # Check if account exists in the database
        self.cur.execute("SELECT COUNT(*) FROM my_project WHERE account_number = %s", (self.account_number,))
        count = self.cur.fetchone()[0]
        return count > 0

        #self.account_exist= check_account_exist()


#Deposite amount
    def deposit(self, account_balance):
        if self.account_exist:
            if account_balance > 0:
                # Fetch the current amount from the database
                self.cur.execute("SELECT account_balance FROM my_project WHERE account_number = %s", (self.account_number,))
                current_amount = self.cur.fetchone()[0]
                
                # Update the balance with the fetched amount and the deposit
                new_balance = current_amount + account_balance
                
                # Update the amount in the database
                self.cur.execute("UPDATE my_project SET account_balance = %s WHERE account_number = %s", (new_balance, self.account_number))
                self.conn.commit()
                print(f"Deposit of ${account_balance} successful. New balance: ${new_balance}")
            else:
                print("Deposit amount must be positive.")
        else:
            print("Account does not exist. Please create an account.")

#Withdraw amount
    def withdraw(self, account_balance):
        if self.account_exist:
            if account_balance > 0:
                # Fetch the current amount from the database
                self.cur.execute("SELECT account_balance FROM my_project WHERE account_number = %s", (self.account_number,))
                current_amount = self.cur.fetchone()[0]
                
                if account_balance <= current_amount:
                    # Calculate new balance after withdrawal
                    new_balance = current_amount - account_balance
                    
                    # Update the amount in the database
                    self.cur.execute("UPDATE my_project SET account_balance = %s WHERE account_number = %s", (new_balance, self.account_number))
                    self.conn.commit()
                    print(f"Withdrawal of ${account_balance} successful. New balance: ${new_balance}")
                else:
                    print("Withdrawal amount exceeds account balance.")
            else:
                print("Withdrawal amount must be positive.")
        else:
            print("Account does not exist. Please create an account.")


#Check the bank balance

    def check_balance(self):
        if self.account_exist:
            # Fetch the amount from the database for the respective account
            self.cur.execute("SELECT account_balance FROM my_project WHERE account_number = %s", (self.account_number,))
            amount = self.cur.fetchone()[0]
            print(f"Account balance: ${amount}")
        else:
            print("Account does not exist. Please create an account.")


#Create new account 
    def create_account(self):
        if not self.account_exist:
            # Insert new account into the database
            self.cur.execute("INSERT INTO my_project (sl_no, account_name,account_number, account_balance) VALUES (%s, %s, %s, %s)",
                             (self.sl_no, self.account_name,self.account_number, self.account_balance))
            self.conn.commit()
            self.account_exist = True
            print("Account created successfully.")
        else:
            print("Account already exists.")
        self.closing()
    def closing(self):
        # Close cursor and connection when object is destroyed
        self.cur.close()
        self.conn.close()

# Step:4 Bank operation

In [ ]:
#Withdraw amount from exist account

In [69]:
acc=Bank_account(2,'Hithesh',1235)
acc.withdraw(500)

Withdrawal of $500 successful. New balance: $4800


In [ ]:
#create new account

In [92]:
acc=Bank_account(4,'Manoj',1237,0)
acc.create_account()

Account already exists.


In [ ]:
#deposite new account

In [96]:
acc=Bank_account(4,'Manoj',1237,0)
acc.deposit(5500)

Deposit of $5500 successful. New balance: $5500
